In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.autograd import Variable
import random
import torch.optim as optim
import pickle
import torch.utils.data
from torch.backends import cudnn
from scipy.sparse import csr_matrix
import math
import bottleneck as bn
import time
import matplotlib.pyplot as plt
import scipy

from Utils.utils import *
from Utils.models import *
from Utils.calibration import *

torch.set_printoptions(precision=4, sci_mode=False)
np.set_printoptions(precision=4, suppress=True)

## Data

In [2]:
gpu = 0
dataset = 'ml10m20'

train_pair = np.load('data/'+dataset+'/train.npy', allow_pickle=True)
train_dic = np.load('data/'+dataset+'/train_dic.npy', allow_pickle=True).item()
val_dic = np.load('data/'+dataset+'/val_dic.npy', allow_pickle=True).item()
trainval_dic = np.load('data/'+dataset+'/trainval_dic.npy', allow_pickle=True).item()
test_dic = np.load('data/'+dataset+'/test_dic.npy', allow_pickle=True).item()

In [3]:
num_user = train_pair[:,0].max() + 1
num_item = int(max(train_pair[:,1].max(), max(np.concatenate([a for a in val_dic.values()])), max(np.concatenate([a for a in test_dic.values()])))) + 1
print(num_user, num_item)
print(train_pair.shape)

69838 8939
(5991023, 2)


## Model

In [4]:
model = torch.load('model/BPR_'+dataset, map_location = 'cuda:'+str(gpu))

Platt_a = np.load('model/A_BPR_'+dataset+'.npy')
Platt_b = np.load('model/B_BPR_'+dataset+'.npy')
print(Platt_a.shape, Platt_b.shape)
print(Platt_a, Platt_b)

(69838,) (69838,)
[1.335  1.3382 0.4524 ... 0.8853 1.3133 1.1053] [-13.5562 -14.2873  -7.187  ...  -7.8323 -14.7685 -10.578 ]


## Test Metrics

In [6]:
N=50

## test
t0 = time.time()
val_loader = torch.utils.data.DataLoader(np.arange(num_user), batch_size=50)
with torch.no_grad():                
    model.eval()
    topk_matrix = np.zeros((num_user, N))
    topk_score_matrix = np.zeros((num_user, N))
    topk_idx_matrix = np.zeros((num_user, N))

    for us in val_loader:
        ## inference
        us_c = us.cuda(gpu)
        t1 = time.time()
        row = model.forward_eval(us_c)
        for idx, u in enumerate(us): # do not recommend interacted items
            row[idx][trainval_dic[u.numpy().item()]] = float('-inf')
            
        ## ranking (sorting)
        topk_score, topk_idx = torch.topk(row, N)
        topk_score_matrix[us] = topk_score.cpu()
        topk_idx_matrix[us] = topk_idx.cpu()

        ## boolean matrix (|U| * N)
        interactions = torch.zeros([us.size()[0], num_item], dtype=torch.bool, device=gpu)
        users_t, items_t = [], []
        for idx, u in enumerate(us):
            u = u.cpu().numpy().item()
            for i in test_dic[u]:
                users_t.append(idx)
                items_t.append(i)        
        interactions[users_t, items_t] = True        
        y_sorted = interactions.gather(-1, topk_idx)
        
        topk_matrix[us] = y_sorted.cpu().numpy()
t_rec = time.time() ##

NDCG, HR, F1 = evaluate(range(1,N+1), topk_matrix, test_dic, num_item, reduce=False)
t_eval = time.time() ##

print('NDCG={}, HR={}, F1={}, time ={:.2f}, {:.2f}'.format(NDCG, HR, F1, t_rec-t0, t_eval-t_rec))

NDCG=[[0.     0.     0.     ... 0.4272 0.4272 0.4272]
 [1.     0.6131 0.7039 ... 0.7616 0.7616 0.7616]
 [0.     0.3869 0.2961 ... 0.1734 0.1734 0.1734]
 ...
 [0.     0.     0.     ... 0.2389 0.2389 0.2389]
 [1.     0.6131 0.4693 ... 0.5224 0.5224 0.5224]
 [1.     1.     1.     ... 0.6011 0.6011 0.6011]], HR=[[0.     0.     0.     ... 1.     1.     1.    ]
 [1.     0.5    0.6667 ... 1.     1.     1.    ]
 [0.     0.5    0.3333 ... 0.1429 0.1429 0.1429]
 ...
 [0.     0.     0.     ... 0.3103 0.3103 0.3103]
 [1.     0.5    0.3333 ... 0.6    0.6    0.6   ]
 [1.     1.     1.     ... 0.5455 0.5455 0.5455]], F1=[[0.     0.     0.     ... 0.1538 0.1509 0.1481]
 [0.4    0.3333 0.5714 ... 0.1538 0.1509 0.1481]
 [0.     0.2222 0.2    ... 0.0364 0.0357 0.0351]
 ...
 [0.     0.     0.     ... 0.2338 0.2308 0.2278]
 [0.1818 0.1667 0.1538 ... 0.2069 0.2034 0.2   ]
 [0.1667 0.3077 0.4286 ... 0.2034 0.2    0.1967]], time =3.59, 3.14


In [8]:
np.mean(F1, axis=0)

array([0.0546, 0.0912, 0.118 , 0.1384, 0.1541, 0.1667, 0.1769, 0.1853, 0.1923, 0.1984,
       0.2032, 0.2073, 0.2105, 0.2133, 0.2157, 0.2178, 0.2194, 0.2207, 0.2219, 0.2228,
       0.2234, 0.224 , 0.2243, 0.2244, 0.2245, 0.2245, 0.2245, 0.2244, 0.2241, 0.2239,
       0.2236, 0.2231, 0.2226, 0.222 , 0.2215, 0.221 , 0.2204, 0.2198, 0.2192, 0.2186,
       0.2179, 0.2172, 0.2166, 0.2159, 0.2152, 0.2145, 0.2138, 0.2132, 0.2125, 0.2117])

# PerK

In [9]:
M = 3000 ## ml10m20

t0 = time.time()
val_loader = torch.utils.data.DataLoader(np.arange(num_user), batch_size=50)
with torch.no_grad():
    model.eval()
    all_sum_matrix_t = np.zeros(num_user)
    val_sum_matrix_t = np.zeros(num_user)
    topk_matrix_t = np.zeros((num_user, M))
    topk_score_matrix = np.zeros((num_user, M))
    topk_idx_matrix = np.zeros((num_user, M))

    for us in val_loader:
        ## inference
        row = model.forward_eval(us.cuda(gpu))
        for idx, u in enumerate(us): # do not recommend interacted items
            user = u.numpy().item()
            val_score = row[idx][val_dic[user]]
            val_sum_matrix_t[user] = torch.sigmoid(torch.mul(val_score, torch.FloatTensor(Platt_a).cuda(gpu)[user]) + torch.FloatTensor(Platt_b).cuda(gpu)[user]).sum().cpu().numpy()
            row[idx][trainval_dic[user]] = float('-inf')

        ## ranking (sorting)
        all_sum_matrix_t[us] = torch.sigmoid(torch.mul(row, torch.FloatTensor(Platt_a).cuda(gpu)[us].resize(len(us), 1)) + torch.FloatTensor(Platt_b).cuda(gpu)[us].resize(len(us), 1)).sum(dim=1).cpu().numpy()

        topk_score, topk_idx = torch.topk(row, M)
        topk_score_matrix[us] = topk_score.cpu()
        topk_idx_matrix[us] = topk_idx.cpu()

        ## boolean matrix (|U| * N)
        interactions = torch.zeros([us.size()[0], num_item], dtype=torch.bool, device=gpu)
        users_t, items_t = [], []
        for idx, u in enumerate(us):
            u = u.cpu().numpy().item()
            for i in test_dic[u]:
                users_t.append(idx)
                items_t.append(i)        
        interactions[users_t, items_t] = True
        y_sorted = interactions.gather(-1, topk_idx)

        topk_matrix_t[us] = y_sorted.cpu().numpy()
        
topk_score_matrix = torch.FloatTensor(topk_score_matrix).cuda(gpu)
NN = num_user
topk_prob_matrix_t = torch.sigmoid(torch.mul(topk_score_matrix, torch.FloatTensor(Platt_a).cuda(gpu)[:NN].resize(NN, 1)) + torch.FloatTensor(Platt_b).cuda(gpu)[:NN].resize(NN, 1))
topk_prob_matrix_t = topk_prob_matrix_t.cpu().numpy()[:, :M]

K = M
ratio_t_K = (topk_prob_matrix_t[:, :K].sum(axis=1) + val_sum_matrix_t) / topk_prob_matrix_t[:, :K].sum(axis=1)

topk_prob_matrix_s = topk_prob_matrix_t * ratio_t_K.reshape(-1, 1)
print(time.time() - t0)

### F1
k = M
S = M
topk_prob_matrix = topk_prob_matrix_s[:, :k]
print(topk_prob_matrix.shape)

Mom_mat = [np.array([(k+s) for s in range(1, S+1)]) for k in range(1,N+1)]

F1_mat = np.zeros((num_user, N))
t0 = time.time()
for u in range(NN):
    if u % 10000 == 0:
        print(u)
    prob = topk_prob_matrix[u]

    F1_S = np.zeros((N, S))
    prob_sum = np.diff(PB_CDF_RNA(prob, np.arange(-1, S))) # (S) vector
    for i in range(N):
        F1_S[i] = 2 * prob_sum * prob[i]

    F1_u = np.cumsum(F1_S, axis=0) # (N * S)
    F1_mat[u] = np.array([np.sum(F1_u[k] / Mom_mat[k]) for k in range(N)])

## PerK
NNN = NN #NN
th_pred = []
for idx, F1_u in enumerate(F1_mat):
    if F1[idx].sum() == 0:
        th_pred.append(-1)
    else:
        th_max = np.argmax(F1_u)
        th_pred.append(th_max + 1)
print(time.time() - t0)

F1_perk = []
for idx, HR_u in enumerate(F1):
    if th_pred[idx] == -1:
        F1_perk.append(HR_u[0])

    else:
        F1_perk.append(HR_u[th_pred[idx]-1])

## Greedy-K train
print('PerK F1: {:.4f}'.format(np.mean(F1_perk)))

/home/wonbin/anaconda3/lib/python3.11/site-packages/torch/_tensor.py:836: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


17.640133380889893
(69838, 3000)
0
10000
20000
30000
40000
50000
60000
132.81352043151855
PerK F1: 0.2538
